# Custom Chatbot Project

The dataset comprises the abstracts of the popular book on [www.goodreads.com](https://www.goodreads.com) for the years 2022 to 2024. This is data that is unknown to the model and thus a good test for the RAG setup.

In [51]:
import openai
import os

openai.api_base = "https://openai.vocareum.com/v1"
# get the api key from the environment variable
openai.api_key = os.getenv("VOC_OPENAI_API_KEY") 

from openai import OpenAI

client = OpenAI(
  api_key=openai.api_key,  
    base_url=openai.api_base
)

## Data Wrangling

In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [52]:

import pandas as pd
import requests
from bs4 import BeautifulSoup

years = ["2022", "2023", "2024"]

# Download the HTML files
for year in years:
    resp = requests.get(f"https://www.goodreads.com/book/popular_by_date/{year}")
    with open(f"goodreads_{year}.html", "w", encoding='utf-8') as f:
        f.write(resp.text)


In [53]:

# read the data
books_abstracts = []
for year in years:
    with open(f"goodreads_{year}.html", "r", encoding='utf-8') as f:
        html = f.read()

    # parse the HTML
    soup = BeautifulSoup(html, 'html.parser')
    for item in soup.find_all('article', class_='BookListItem'):

        title = item.find('h3', class_='Text Text__title3 Text__umber').text
        authors = item.find('div', class_='BookListItem__authors').text
        abstract = item.find('div', class_='TruncatedContent').text
        books_abstracts.append(f"{title} by {authors} ({year})\n{abstract}")


# save the data
df = pd.DataFrame(books_abstracts, columns=["text"])
df.to_csv("goodreads_books.csv", index=False)

print(f"Number of books: {len(df)}")

print(df.head())

Number of books: 45
                                                text
0  The Housemaid (The Housemaid, #1) by Freida Mc...
1  It Starts with Us (It Ends with Us, #2) by Col...
2  Reminders of Him by Colleen Hoover (2022)\nA t...
3  Book Lovers by Emily Henry (2022)\nOne summer....
4  Tomorrow, and Tomorrow, and Tomorrow by Gabrie...


In [54]:
# code taken from the class notebook casestudy.ipynb
# Create a list to store the embeddings
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 100
embeddings = []
for i in range(0, len(df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = client.embeddings.create(
        input=df.iloc[i:i+batch_size]["text"].tolist(),
        model=EMBEDDING_MODEL_NAME
    )
    
    # Add embeddings to list
    embeddings.extend([data.embedding for data in response.data])

# Add embeddings list to dataframe
df["embeddings"] = embeddings
df.to_csv("goodreads_books_embedded.csv", index=False)
df.head()

,text,embeddings
0,"The Housemaid (The Housemaid, #1) by Freida Mc...","[-0.009157824330031872, -0.02493787370622158, ..."
1,"It Starts with Us (It Ends with Us, #2) by Col...","[0.0013730453792959452, -0.003243256825953722,..."
2,Reminders of Him by Colleen Hoover (2022)\nA t...,"[-0.00190580019261688, -0.01718563586473465, -..."
3,Book Lovers by Emily Henry (2022)\nOne summer....,"[-0.010121977888047695, -0.017106615006923676,..."
4,"Tomorrow, and Tomorrow, and Tomorrow by Gabrie...","[0.013451321050524712, -0.022161180153489113, ..."


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [55]:
# code taken from the class notebook casestudy.ipynb and from https://github.com/openai/openai-python/blob/release-v0.28.0/openai/embeddings_utils.py
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def distances_from_embeddings(query_embedding,embeddings):
    distances = [
        1-cosine_similarity(query_embedding, embedding)
        for embedding in embeddings
    ]
    return distances

def get_embedding(text, engine=EMBEDDING_MODEL_NAME):
    return client.embeddings.create(
        input=[text],
        model=engine
    ).data[0].embedding

def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    
    # Get embeddings for the question text
    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)
    
    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
    )
    
    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy

In [56]:
ranking = get_rows_sorted_by_relevance("Books by Britney", df)
ranking.head()

,text,embeddings,distances
28,The Woman in Me by Britney Spears (2023)\nThe ...,"[-0.01765580102801323, -0.022641273215413094, ...",0.158990
9,Never Lie by Freida McFadden (2022)\nNewlyweds...,"[-0.020385894924402237, -0.011535530909895897,...",0.218110
3,Book Lovers by Emily Henry (2022)\nOne summer....,"[-0.010121977888047695, -0.017106615006923676,...",0.218744
6,I'm Glad My Mom Died by Jennette McCurdy (2022...,"[-0.016894932836294174, -0.0012957364087924361...",0.221718
19,None of This Is True by Lisa Jewell (2023)\nLi...,"[0.004881617147475481, 0.0009463515016250312, ...",0.223934


In [57]:
# based on the class notebook casestudy.ipynb 

import tiktoken

def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    # Count the number of tokens in the prompt template and question
    prompt_template = """
You are an expert for popular books published in the years 2022, 2023 and 2024.
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

{}

---

Question: {}
Answer:"""
    
    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))
    
    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:
        
        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count
        
        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)

In [58]:
# based on the class notebook casestudy.ipynb 

COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question(
    question, df, max_prompt_tokens=1800, max_answer_tokens=150
):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model
    
    If the model produces an error, return an empty string
    """
    
    prompt = create_prompt(question, df, max_prompt_tokens)
    
    try:
        response = client.completions.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return response.choices[0].text.strip()
    except Exception as e:
        print(e)
        return ""
    
def answer_question_with_model_only(
    question, max_answer_tokens=150
):
   
    try:
        response = client.completions.create(
            model=COMPLETION_MODEL_NAME,
            prompt="You are an expert for popular books published in the years 2022, 2023 and 2024. Answer the following question: " + question,
            max_tokens=max_answer_tokens
        )
        return response.choices[0].text.strip()
    except Exception as e:
        print(e)
        return ""

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [59]:
answer_question("Can you reccomend a books with fantasy and magic?", df)

'Yes, I would highly recommend Divine Rivals (Letters of Enchantment, #1) by Rebecca Ross and The Serpent and the Wings of Night (Crowns of Nyaxia, #1) by Carissa Broadbent. These books both have elements of fantasy and magic in their plot and have received critical acclaim as popular books published in 2023 and 2022 respectively.'

In [60]:
# run with an empty dataframe
answer_question("Can you reccomend a books with fantasy and magic?", df[0:0])

"I don't know."

In [61]:
answer_question_with_model_only("Can you reccomend a books with fantasy and magic?")

'Sure! I would recommend "The Blood of Stars" by Elizabeth Lim, published in 2022. It follows a young girl named Maia who possesses powerful magic that she must learn to control in order to save her kingdom from dark forces. Another great read is "The Court of Miracles" by Kester Grant, published in 2023. This book is set in an alternate Paris where magic and fantasy elements are seamlessly woven into the story as a young girl navigates through a criminal underworld to save her sister. Lastly, I would suggest "The Last Magician" by Lisa Maxwell, published in 2024. It follows a girl named Esta who has the ability to time travel and must use her powers to save a magical artifact'

The listed books are either from previous years or invented.

### Question 2

In [62]:
answer_question("Did Britney write any books in 2023?", df)

'Yes, Britney Spears released her book "The Woman in Me" in 2023.'

In [63]:
# run with an empty dataframe
answer_question("Did Britney write any books in 2023?", df[0:0])

"I don't know"

In [64]:
answer_question_with_model_only("Did Britney write any books in 2023?")

'As an AI, I do not have access to current or future information. I am unable to confirm if Britney wrote any books in 2023.'